In [1]:
import illustris_python as il
import numpy as np
import h5py
from groupcat import gcPath, offsetPath

In [2]:
from importlib import reload  

In [3]:
from findAccTags import loadIDsSubhalos, loadIDsGroups, getMergerTrees

In [4]:
basePath = '/home/tnguser/sims.TNG/TNG100-1/output'
dataPath = '../Particles-TNG-100/'
snapNum0 = 99         
partType=4                 # at the level of stars
h0= 0.6774
stype='Subhalo'            # study subhalos for now.

## Find a List of MW-mass haloes at redshift z=0

In [4]:
GroupFirstSub = il.groupcat.loadHalos(basePath,snapNum0,fields=['GroupFirstSub'])

fields = ['SubhaloMassType',]
subhalos = il.groupcat.loadSubhalos(basePath,snapNum0,fields=fields)
dm=subhalos[:,1]*(1e10/h0)
sm=subhalos[:,4]*(1e10/h0)
mask=((dm>0.8e12) & (dm<2e12)) & ((sm>2e10) & (sm<1e11))
arg_list_prospective=np.where(mask)[0]

arg=np.in1d(arg_list_prospective,GroupFirstSub)
subfindID_prospective=arg_list_prospective[arg]

KeyboardInterrupt: 

In [5]:
subfindID0=subfindID_prospective[1]

## Find Tags of a Subhalo given by its SubfindID

In [5]:
subfindID0=383118

In [6]:
# Find particle IDs of subhalo at z=0
particleIDs=il.snapshot.loadSubhalo(basePath, snapNum0, subfindID0, partType, fields=['ParticleIDs',])

In [7]:
# get merger tree of galaxy
tree, mpb, fof_branch, ext_branch =  getMergerTrees(subfindID0, basePath, snapNum0)

# where mpb, fof_branch and ext_branch refer to the indices of tree
# for instance tree['SubfindID'][mpb[0]] is the first SubfindID of the mpb.
# furthermore, Tree contains a lot of fields including
# 'SubhaloID','NextProgenitorID','MainLeafProgenitorID',
# 'FirstProgenitorID','LastProgenitorID','SubhaloMassType',
# 'SnapNum','SubfindID','FirstSubhaloInFOFGroupID','SubhaloGrNr']

In [29]:
# Mask for storing and registering where the particle was born.
particleIDs_mpb=np.zeros_like(particleIDs,dtype=np.bool)
particleIDs_fof=np.zeros_like(particleIDs,dtype=np.bool)
particleIDs_ext=np.zeros_like(particleIDs,dtype=np.bool)

# check if already born.
particleIDs_pre=np.zeros_like(particleIDs,dtype=np.bool)

Algorithm consists of characterizing the birth place of the stellar particles into 3 zones (mpb, fof and ext).
To do this effectively, we use boolean masks to store the stellar particles particles found in each of the three zones, 
as well as to store previously born stellar particles.

ote, we identify the particles using masks and the in1d function.

In [30]:
# The present algorithm only considers Subhaloes. More complicated rules can be constructed.

for Snap0 in np.unique(tree['SnapNum'][mpb]): 
    if Snap0>4:
        # MPB
        arg0=np.where(tree['SnapNum'][mpb]==Snap0)[0]
        SubfindID=tree['SubfindID'][mpb][arg0]
        particleIDs0=loadIDsSubhalos(SubfindID,Snap0,dataPath)
        if len(particleIDs0)>0:
            particleIDs0_present=np.in1d(particleIDs,particleIDs0, assume_unique=True)
            particleIDs_mpb=np.logical_or(particleIDs_mpb,np.greater(particleIDs0_present,particleIDs_pre))
        
        # FOF
        arg1=np.where(tree['SnapNum'][fof_branch]==Snap0)[0]
        SubfindIDs=tree['SubfindID'][fof_branch][arg1]
        particleIDs1=loadIDsSubhalos(SubfindIDs,Snap0,dataPath)
        if len(particleIDs1)>0:
            particleIDs1_present=np.in1d(particleIDs,particleIDs1, assume_unique=True)
            particleIDs_fof=np.logical_or(particleIDs_fof,np.greater(particleIDs1_present,particleIDs_pre))
        
        # EXT
        arg2=np.where(tree['SnapNum'][ext_branch]==Snap0)[0]
        SubfindIDs=tree['SubfindID'][ext_branch][arg2]
        particleIDs2=loadIDsSubhalos(SubfindIDs,Snap0,dataPath)
        if len(particleIDs2)>0:
            particleIDs2_present=np.in1d(particleIDs,particleIDs2, assume_unique=True)
            particleIDs_ext=np.logical_or(particleIDs_ext,np.greater(particleIDs2_present,particleIDs_pre))
            
            
        # store if particle already exists.
        particleIDs_pre=np.logical_or(particleIDs_pre, particleIDs0_present)
        particleIDs_pre=np.logical_or(particleIDs_pre, particleIDs1_present)
        particleIDs_pre=np.logical_or(particleIDs_pre, particleIDs2_present)
        

In [32]:
print(len(np.where(particleIDs_mpb)[0]))
print(len(np.where(particleIDs_fof)[0]))
print(len(np.where(particleIDs_ext)[0]))

25949
1357
1366


In [33]:
len(particleIDs)

28672

In [34]:
25949+1357+1366

28672